In [1]:
import numpy as np
import os
import datetime
import time
#import argparse


%matplotlib inline 
import matplotlib.pyplot as plt


In [2]:
pwd

'C:\\Users\\stefa\\Desktop\\esp_cube-main'

In [3]:
#Backup
#parser = argparse.ArgumentParser()
#parser.add_argument("-l", "--lato", help="Lato della Box",type=int)
#parser.add_argument("-p", "--points", help="Number of Points",type=int)
#parser.add_argument("-o", "--output", help="Output")
#args = parser.parse_args()
#lato_box=args.lato
#npoints=args.points

In [4]:
bohr=0.529177249 # bohr to angstrom

In [5]:
input_file=input("Please insert a PDB file: ")
lato_box=int(input("Please insert box side: "))
npoints=int(input("Please decleare how many points you want to consider: "))
output=input("Output's name: ")

Please insert a PDB file: test_9.pdb
Please insert box side: 20
Please decleare how many points you want to consider: 20
Output's name: prova_9.cube


In [6]:
spacing=((lato_box*bohr)-(-lato_box*bohr))/9/bohr # spacing

In [7]:
class Atoms:
    def __init__(self,x,y,z,charge):
        self.x=x
        self.y=y
        self.z=z
        self.charge=charge

In [8]:
with open(input_file) as file:
    atoms = []
    for line in file:
        fields = line.strip().split()
        if fields[0]=='ATOM':
            x=float(fields[5])
            y=float(fields[6])
            z=float(fields[7])
            charge=float(fields[8])
            atoms.append(Atoms(x,y,z,charge))
        else:
            pass

In [11]:
X = np.linspace(-(lato_box)*bohr,lato_box*bohr,npoints) # min, max, npoints
Y = np.linspace(-(lato_box)*bohr,lato_box*bohr,npoints)
Z = np.linspace(-(lato_box)*bohr,lato_box*bohr,npoints)


In [12]:
xx, yy, zz = np.meshgrid(X,Y,Z)

In [13]:
iterator=0
for i in atoms:
    if iterator==0:
        a=i.charge/np.sqrt((xx.flatten()-i.x)**2+(yy.flatten()-i.y)**2+(zz.flatten()-i.y)**2)
        iterator=+1
    else:
        a=a+i.charge/np.sqrt((xx.flatten()-i.x)**2+(yy.flatten()-i.y)**2+(zz.flatten()-i.y)**2)
    

In [14]:
a=a.reshape(100,-1)

In [15]:
a=a.reshape(-1,npoints)

In [16]:
fmt=np.array(['  %3.4e'] *npoints)
fmt

array(['  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e',
       '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e',
       '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e', '  %3.4e',
       '  %3.4e', '  %3.4e'], dtype='<U7')

In [17]:
fmt2=""
iterator=1
for i in fmt:
    fmt2=fmt2+str(i)
    if iterator % 6 == 0:
        fmt2=fmt2+str('\n')
    iterator=iterator+1
print(fmt2)


  %3.4e  %3.4e  %3.4e  %3.4e  %3.4e  %3.4e
  %3.4e  %3.4e  %3.4e  %3.4e  %3.4e  %3.4e
  %3.4e  %3.4e  %3.4e  %3.4e  %3.4e  %3.4e
  %3.4e  %3.4e


In [18]:
spacing=format(spacing, '.5f')
lato_box=format(lato_box, '.5f')

In [19]:
header=""" Opt job potenital=scf
Electrostatic potential from Total SCF Density
    {}  -{}   -{}   -{}
   10    {}     0.00000     0.00000
   10    0.00000     {}     0.00000
   10    0.00000     0.00000     {}""".format(len(atoms),lato_box,lato_box,lato_box,spacing,spacing,spacing)
for i in atoms:
    primo=format(i.x/bohr, '.5f')
    secondo=format(i.y/bohr, '.5f')
    terzo=format(i.z/bohr, '.5f')
    header=header+"\n    0    0.00000     {}     {}    {}".format(primo,secondo,terzo)


In [20]:
np.savetxt('{}'.format(output),a,fmt=fmt2,header=header,comments='')
